In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/first-gop-debate-twitter-sentiment/Sentiment.csv
/kaggle/input/first-gop-debate-twitter-sentiment/database.sqlite


# **Import LIBS**

In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D

from keras.utils.np_utils import to_categorical

import re

# **Load Data set**

In [3]:
Dataset = pd.read_csv('/kaggle/input/first-gop-debate-twitter-sentiment/Sentiment.csv')

In [4]:
Dataset.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


# **Data Preprossing**

In [5]:
Dataset = Dataset[['text', 'sentiment']]

In [6]:
Dataset.head()


,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [7]:
Dataset['text']


0        RT @NancyLeeGrahn: How did everyone feel about...
1        RT @ScottWalker: Didn't catch the full #GOPdeb...
2        RT @TJMShow: No mention of Tamir Rice and the ...
3        RT @RobGeorge: That Carly Fiorina is trending ...
4        RT @DanScavino: #GOPDebate w/ @realDonaldTrump...
                               ...                        
13866    RT @cappy_yarbrough: Love to see men who will ...
13867    RT @georgehenryw: Who thought Huckabee exceede...
13868    RT @Lrihendry: #TedCruz As President, I will a...
13869    RT @JRehling: #GOPDebate Donald Trump says tha...
13870    RT @Lrihendry: #TedCruz headed into the Presid...
Name: text, Length: 13871, dtype: object

In [8]:
Dataset = Dataset[Dataset.sentiment !="Neutral"] 


In [9]:
Dataset['text'] = Dataset['text'].map(lambda x: x.lower())
Dataset['text'] = Dataset['text'].map(lambda x: re.sub('[^a-z0-9\s]','', x))
Dataset['text']=Dataset['text'].str.replace('rt',' ')


# for i in range(Dataset.shape[0]):
#     Dataset['text'][i] = Dataset['text'][i].replace('rt', '')

In [10]:
Dataset['text']

1          scottwalker didnt catch the full gopdebate l...
3          robgeorge that carly fiorina is trending  ho...
4          danscavino gopdebate w realdonaldtrump deliv...
5          gregabbotttx tedcruz on my first day i will ...
6          warriorwoman91 i liked her and was happy whe...
                               ...                        
13866      cappyyarbrough love to see men who will neve...
13867      georgehenryw who thought huckabee exceeded t...
13868      lrihendry tedcruz as president i will always...
13869      jrehling gopdebate donald trump says that he...
13870      lrihendry tedcruz headed into the presidenti...
Name: text, Length: 10729, dtype: object

In [11]:
max_features = 2000

tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(Dataset['text'].values)
X = tokenizer.texts_to_sequences(Dataset['text'].values)
X = pad_sequences(X)
X

array([[   0,    0,    0, ..., 1317, 1403,  742],
       [   0,    0,    0, ...,  233,  723,   17],
       [   0,    0,    0, ...,  207,  371,  670],
       ...,
       [   0,    0,    0, ...,   72,   65,    3],
       [   0,    0,    0, ..., 1018, 1417,   73],
       [   0,    0,    0, ...,  197,    3,  721]], dtype=int32)

In [12]:
Dataset['text']


1          scottwalker didnt catch the full gopdebate l...
3          robgeorge that carly fiorina is trending  ho...
4          danscavino gopdebate w realdonaldtrump deliv...
5          gregabbotttx tedcruz on my first day i will ...
6          warriorwoman91 i liked her and was happy whe...
                               ...                        
13866      cappyyarbrough love to see men who will neve...
13867      georgehenryw who thought huckabee exceeded t...
13868      lrihendry tedcruz as president i will always...
13869      jrehling gopdebate donald trump says that he...
13870      lrihendry tedcruz headed into the presidenti...
Name: text, Length: 10729, dtype: object

In [13]:
embedding_dim = 100
lstm_out = 130

model = Sequential()

model.add(Embedding(max_features, embedding_dim, input_length=X.shape[1]))

model.add(SpatialDropout1D(0.4))

model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.3))

model.add(Dense(2, activation='softmax')) 




In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# **Splitting**

In [15]:
y = pd.get_dummies(Dataset['sentiment']).values

In [16]:
x_train,x_test,y_train,y_test =train_test_split(X,y,test_size=.2,random_state=42)

In [17]:
BATCH_SIZE = 32
EPOCHS = 8

model.fit(x_train, y_train, 
                 epochs = EPOCHS, 
                 validation_data = (x_test, y_test), 
                 batch_size = BATCH_SIZE)

Epoch 1/8
269/269 [==============================] - 20s 62ms/step - loss: 0.4298 - accuracy: 0.8193 - val_loss: 0.3618 - val_accuracy: 0.8462
Epoch 2/8
269/269 [==============================] - 16s 58ms/step - loss: 0.3142 - accuracy: 0.8689 - val_loss: 0.3427 - val_accuracy: 0.8541
Epoch 3/8
269/269 [==============================] - 16s 61ms/step - loss: 0.2762 - accuracy: 0.8849 - val_loss: 0.3440 - val_accuracy: 0.8579
Epoch 4/8
269/269 [==============================] - 16s 60ms/step - loss: 0.2517 - accuracy: 0.8988 - val_loss: 0.3526 - val_accuracy: 0.8458
Epoch 5/8
269/269 [==============================] - 16s 59ms/step - loss: 0.2258 - accuracy: 0.9080 - val_loss: 0.3652 - val_accuracy: 0.8490
Epoch 6/8
269/269 [==============================] - 16s 60ms/step - loss: 0.2123 - accuracy: 0.9122 - val_loss: 0.3972 - val_accuracy: 0.8481
Epoch 7/8
269/269 [==============================] - 16s 58ms/step - loss: 0.1967 - accuracy: 0.9173 - val_loss: 0.4565 - val_accuracy: 0.8439

In [18]:
score, acc = model.evaluate(x_test, y_test, batch_size = BATCH_SIZE)

print('Score: {:.2f}'.format(score))
print('Accuracy: {:.2f}'.format(acc))

68/68 [==============================] - 1s 12ms/step - loss: 0.4519 - accuracy: 0.8304
Score: 0.45
Accuracy: 0.83


In [22]:
# Let's test the model 

tweet = ['it is very good ']

# Vectorizing the tweet by the pre-fitted tokenizer instance
tweet = tokenizer.texts_to_sequences(tweet)

# Padding the tweet to have exactly the same shape as `embedding_2` input
tweet = pad_sequences(tweet, maxlen=28, dtype='int32', value=0)

sentiment = model.predict(tweet, batch_size=1, verbose=1)[0]

if(np.argmax(sentiment) == 0):
    print("negative")
    
elif (np.argmax(sentiment) == 1):
    print("positive")

1/1 [==============================] - 0s 29ms/step
positive
